In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

status = pd.read_table("data/status.csv", sep = ",")
weather = pd.read_table("data/weather.csv", sep = ",")

In [2]:
print(status.shape)
status.head()

(1226400, 8)


,id,year,month,day,hour,station_id,bikes_available,predict
0,0,2013,9,1,0,0,11.0,0
1,1,2013,9,1,1,0,11.0,0
2,2,2013,9,1,2,0,11.0,0
3,3,2013,9,1,3,0,11.0,0
4,4,2013,9,1,4,0,11.0,0


In [3]:
print(weather.shape)
weather.head()

(730, 22)


,date,max_temperature,mean_temperature,min_temperature,max_dew_point,mean_dew_point,min_dew_point,max_humidity,mean_humidity,min_humidity,...,min_sea_level_pressure,max_visibility,mean_visibility,min_visibility,max_wind_Speed,mean_wind_speed,precipitation,cloud_cover,events,wind_dir_degrees
0,2013-09-01,81,70,61,62,58,54,80,67,47,...,29.85,10,10,10,14,4,0.00,1,NaN,354
1,2013-09-02,80,71,66,64,61,58,80,70,58,...,29.86,10,10,10,14,4,0.00,5,NaN,337
2,2013-09-03,81,69,58,60,56,52,82,65,44,...,29.93,10,10,10,19,2,1.71,6,Rain,341
3,2013-09-04,82,68,56,61,55,49,81,64,43,...,29.94,10,10,10,15,0,0.00,0,NaN,324
4,2013-09-05,81,68,56,59,54,50,81,63,41,...,29.95,10,10,10,16,1,0.00,0,NaN,335


In [4]:
# 曜日の取得　

import datetime as dt

dd = pd.DataFrame(status, columns = ['year', 'month', 'day'])
dd = pd.to_datetime(dd, format = '%Y%m%d')
status['day_name'] = dd.dt.day_name()

In [5]:
# 祝日ダミー

import holidays
us_holidays = holidays.US()
status["holiday"] = [
    1 if str(val).split()[0] in us_holidays else 0 for val in dd
]

In [6]:
# 平均気温，雨ダミー

# weather['events'].value_counts()
temp = weather.mean_temperature[0]*(dt.datetime.strptime(weather.date[0], '%Y-%m-%d').date() == dd.dt.date)
rain = 1*(weather.events[0] in ['Rain', 'Fog', 'Fog-Rain'])*(dt.datetime.strptime(weather.date[0], '%Y-%m-%d').date() == dd.dt.date)

for i in range(1,len(weather)):
    temp = temp + weather.mean_temperature[i]*(dt.datetime.strptime(weather.date[i], '%Y-%m-%d').date() == dd.dt.date)
    rain = rain + 1*(weather.events[i] in ['Rain', 'Fog', 'Fog-Rain'])*(dt.datetime.strptime(weather.date[i], '%Y-%m-%d').date() == dd.dt.date)

status['temp'] = temp
status['rain'] = rain

In [7]:
status['rain'].value_counts()

0    1068480
1     157920
Name: rain, dtype: int64

In [8]:
# 前日の同じ時間，前週の同じ曜日の同じ時間のbikes_available

ddt = pd.DataFrame(status, columns = ['year', 'month', 'day', 'hour'])
ddt = pd.to_datetime(ddt, format = '%Y%m%d%H')
status['datet'] = ddt

ddt_yd = ddt - dt.timedelta(days = 1) # 1日前
df_yd = pd.DataFrame({'datet': ddt_yd})

ddt_pw = ddt - dt.timedelta(weeks = 1) # 1週間前
df_pw = pd.DataFrame({'datet': ddt_pw})

In [17]:
# add work days
status["workday"] = np.where((status["holiday"] == 0) & (status["day_name"] != "Saturday") & (status["day_name"] != "Sunday"), 1, 0)

In [9]:
data_id = status[status.station_id == 0]
ddt_yd_id = df_yd[status.station_id == 0]
ddt_pw_id = df_pw[status.station_id == 0]
ddt_yd = pd.merge(ddt_yd_id, data_id.loc[:, ['datet', 'bikes_available']], on = 'datet', how = 'left')
ddt_pw = pd.merge(ddt_pw_id, data_id.loc[:, ['datet', 'bikes_available']], on = 'datet', how = 'left')

# 欠損値は曜日ごとの平均値で埋める

mean_day = data_id.groupby(data_id['datet'].dt.day_name())['bikes_available'].mean()

ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Friday', 'bikes_available']    = ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Friday', 'bikes_available'].fillna(mean_day['Friday'])
ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Monday', 'bikes_available']    = ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Monday', 'bikes_available'].fillna(mean_day['Monday'])
ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Saturday', 'bikes_available']  = ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Saturday', 'bikes_available'].fillna(mean_day['Saturday'])
ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Sunday', 'bikes_available']    = ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Sunday', 'bikes_available'].fillna(mean_day['Sunday'])
ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Thursday', 'bikes_available']  = ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Thursday', 'bikes_available'].fillna(mean_day['Thursday'])
ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Tuesday', 'bikes_available']   = ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Tuesday', 'bikes_available'].fillna(mean_day['Tuesday'])
ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Wednesday', 'bikes_available'] = ddt_yd.loc[ddt_yd['datet'].dt.day_name() == 'Wednesday', 'bikes_available'].fillna(mean_day['Wednesday'])

ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Friday', 'bikes_available']    = ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Friday', 'bikes_available'].fillna(mean_day['Friday'])
ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Monday', 'bikes_available']    = ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Monday', 'bikes_available'].fillna(mean_day['Monday'])
ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Saturday', 'bikes_available']  = ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Saturday', 'bikes_available'].fillna(mean_day['Saturday'])
ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Sunday', 'bikes_available']    = ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Sunday', 'bikes_available'].fillna(mean_day['Sunday'])
ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Thursday', 'bikes_available']  = ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Thursday', 'bikes_available'].fillna(mean_day['Thursday'])
ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Tuesday', 'bikes_available']   = ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Tuesday', 'bikes_available'].fillna(mean_day['Tuesday'])
ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Wednesday', 'bikes_available'] = ddt_pw.loc[ddt_pw['datet'].dt.day_name() == 'Wednesday', 'bikes_available'].fillna(mean_day['Wednesday'])


In [10]:
for i in range(1, 70):
    data_id = status[status.station_id == i]
    ddt_yd_id = df_yd[status.station_id == i]
    ddt_pw_id = df_pw[status.station_id == i]
    ddt_yd_m  = pd.merge(ddt_yd_id, data_id.loc[:, ['datet', 'bikes_available']], on = 'datet', how = 'left')
    ddt_pw_m  = pd.merge(ddt_pw_id, data_id.loc[:, ['datet', 'bikes_available']], on = 'datet', how = 'left')
    
    mean_day = data_id.groupby(data_id['datet'].dt.day_name())['bikes_available'].mean()
    
    ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Friday', 'bikes_available']    = ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Friday', 'bikes_available'].fillna(mean_day['Friday'])
    ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Monday', 'bikes_available']    = ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Monday', 'bikes_available'].fillna(mean_day['Monday'])
    ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Saturday', 'bikes_available']  = ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Saturday', 'bikes_available'].fillna(mean_day['Saturday'])
    ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Sunday', 'bikes_available']    = ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Sunday', 'bikes_available'].fillna(mean_day['Sunday'])
    ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Thursday', 'bikes_available']  = ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Thursday', 'bikes_available'].fillna(mean_day['Thursday'])
    ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Tuesday', 'bikes_available']   = ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Tuesday', 'bikes_available'].fillna(mean_day['Tuesday'])
    ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Wednesday', 'bikes_available'] = ddt_yd_m.loc[ddt_yd_m['datet'].dt.day_name() == 'Wednesday', 'bikes_available'].fillna(mean_day['Wednesday'])

    ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Friday', 'bikes_available']    = ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Friday', 'bikes_available'].fillna(mean_day['Friday'])
    ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Monday', 'bikes_available']    = ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Monday', 'bikes_available'].fillna(mean_day['Monday'])
    ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Saturday', 'bikes_available']  = ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Saturday', 'bikes_available'].fillna(mean_day['Saturday'])
    ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Sunday', 'bikes_available']    = ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Sunday', 'bikes_available'].fillna(mean_day['Sunday'])
    ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Thursday', 'bikes_available']  = ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Thursday', 'bikes_available'].fillna(mean_day['Thursday'])
    ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Tuesday', 'bikes_available']   = ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Tuesday', 'bikes_available'].fillna(mean_day['Tuesday'])
    ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Wednesday', 'bikes_available'] = ddt_pw_m.loc[ddt_pw_m['datet'].dt.day_name() == 'Wednesday', 'bikes_available'].fillna(mean_day['Wednesday'])
    
    ddt_yd = ddt_yd.append(ddt_yd_m)
    ddt_pw = ddt_pw.append(ddt_pw_m)

In [11]:
ddt_yd = ddt_yd.set_index(status.id)
ddt_pw = ddt_pw.set_index(status.id)

In [12]:
lag_bs_avail = pd.concat([ddt_yd, ddt_pw], axis=1)

In [13]:
status = pd.merge(status, lag_bs_avail, left_index = True, right_index = True)

In [14]:
train = status[status.predict == 0]
test  = status[status.predict == 1]
print(train.shape, test.shape)
test.head()

(1033200, 17) (193200, 17)


,id,year,month,day,hour,station_id,bikes_available_x,predict,day_name,holiday,temp,rain,datet_x,datet_y,bikes_available_y,datet_y,bikes_available_y
8761,8761,2014,9,1,1,0,NaN,1,Monday,1,73,0,2014-09-01 01:00:00,2014-08-31 01:00:00,11.0,2014-08-25 01:00:00,17.0
8762,8762,2014,9,1,2,0,NaN,1,Monday,1,73,0,2014-09-01 02:00:00,2014-08-31 02:00:00,11.0,2014-08-25 02:00:00,17.0
8763,8763,2014,9,1,3,0,NaN,1,Monday,1,73,0,2014-09-01 03:00:00,2014-08-31 03:00:00,11.0,2014-08-25 03:00:00,17.0
8764,8764,2014,9,1,4,0,NaN,1,Monday,1,73,0,2014-09-01 04:00:00,2014-08-31 04:00:00,11.0,2014-08-25 04:00:00,17.0
8765,8765,2014,9,1,5,0,NaN,1,Monday,1,73,0,2014-09-01 05:00:00,2014-08-31 05:00:00,10.0,2014-08-25 05:00:00,17.0


In [18]:
train.to_csv('data/train.csv')
test.to_csv('data/test.csv')